In [2]:
import pandas as pd
import dask.dataframe as dd
import os
import re

root = '/data/khodadaa/wiki09-pagecount-logs/dumps/'
pagecount_save_path = '/data/khodadaa/wiki09-pagecount-logs/wiki09_pagecounts_dumps.csv'

In [ ]:
%%time

# Merge files containing only images with chunking the data
df_pagecounts = pd.DataFrame(columns=['lang', 'article'])
files = os.listdir(root)
files.sort()
print('Files: ', files)
i = 0;
exceptions = []
for f in files:
    if f.startswith('pagecounts-'):
        i += 1
        print(i, f)
        try:
            iter_csv = pd.read_csv(root+f,
                             usecols=[0,1,2],
                             dtype= {0:str, 1:str, 2:int},
                             header=None,
                             sep='\s+',
                             encoding='latin1',
                             iterator=True,
                             chunksize=10000)
            df = pd.concat([chunk[chunk[0].str.lower().str.startswith('en') &
                                  chunk[1].str.lower().str.endswith(('.svg','.jpg','.jpeg','.png','.gif'))]
                            for chunk in iter_csv])
            df.rename(columns={0: 'lang', 1:'article', 2:f}, inplace=True)
            df_pagecounts = df_pagecounts.merge(df, on=['lang', 'article'], how='outer')
        except Exception as e:
            print(e)
            exceptions += [f]
        if i%240 == 0:
            df_pagecounts.to_csv(pagecount_save_path[:-4]+f[10:]+'.csv', index=False)
            df_pagecounts = pd.DataFrame(columns=['lang', 'article'])
df_pagecounts.to_csv(pagecount_save_path, index=False)
print(exceptions)


# Merge files containing all wiki pages without chunking the data
# df_pagecounts = pd.DataFrame(columns=['lang', 'article'])
# files = os.listdir(root)
# files.sort()
# print('Files: ', files)
# i = 0;
# exceptions = []
# for f in files:
#     if f.startswith('pagecounts-'):        
#         i += 1
#         print(i, f)
#         try:
#             df = pd.read_csv(root+f,
#                              usecols=[0,1,2],                 
#                              dtype= {0:str, 1:str, 2:int},
#                              header=None,
#                              sep='\s+', 
#                              encoding='latin1')
#             df.rename(columns={0: 'lang', 1:'article', 2:f}, inplace=True)        
#             df_pagecounts = pd.merge(df_pagecounts, df, on=['lang', 'article'], how='outer')            
#         except e:
#             print(e)
#             exceptions += [f]
#         if i%500 == 3:
#             df_pagecounts.to_csv(pagecount_save_path, index=False)        
# df_pagecounts.to_csv(pagecount_save_path, index=False)
# print(exceptions)

In [ ]:
df_pagecounts = pd.read_csv(pagecount_save_path)

# cite: http://pandas.pydata.org/pandas-docs/stable/text.html#text-string-methods
regex_pat = re.compile(r'.*\.svg|.*\.jpg|.*\.jpeg|.*\.png|.*\.gif', re.IGNORECASE)
img_search = df_pagecounts[df_pagecounts.article.str.contains(regex_pat, na=False)]
# img_match = df_pagecounts[df_pagecounts.article.str.match(regex_pat, na=False)]

In [ ]:
%%bash
head /data/khodadaa/wiki09-pagecount-logs/pagecounts-20090101-010000

In [ ]:
%%bash
sed -n '19969,19969p;19973q' /data/khodadaa/wiki09-pagecount-logs/pagecounts-20090101-010000

In [ ]:
print(df.loc[19968, 'article'])

# Import dataframe in chunks

In [53]:
%%time
iter_csv = pd.read_csv('/data/khodadaa/wiki09-pagecount-logs/wiki09_pagecounts_dumpspagecounts-20090102-230000.csv', 
                       dtype={'lang': str, 'article':str}, iterator=True, chunksize=10000)
df3 = pd.concat([chunk[chunk['lang'].str.startswith('en') & chunk['article'].str.endswith(('.svg','.jpg','.jpeg','.png','.gif'))] 
                 for chunk in iter_csv])

CPU times: user 2min 8s, sys: 2.89 s, total: 2min 11s
Wall time: 2min 11s


In [47]:
%%time
iter_csv = pd.read_csv('/data/khodadaa/wiki09-pagecount-logs/dumps/pagecounts-20090101-090000',
    usecols=[0,1,2],                 
    dtype= {0:str, 1:str, 2:int},
    header=None,
    sep='\s+', 
    encoding='latin1',
    iterator=True, 
    chunksize=1000)
df4 = pd.concat([chunk[chunk[0].str.lower().str.startswith('en') & 
                       chunk[1].str.lower().str.endswith(('.svg','.jpg','.jpeg','.png','.gif'))] 
                 for chunk in iter_csv])
df4.rename(columns={0: 'lang', 1:'article', 2:'count'}, inplace=True)

CPU times: user 19.9 s, sys: 110 ms, total: 20 s
Wall time: 20 s


In [46]:
df4.head()

,lang,article,count
829281,en.b,Adventist_Youth_Honors_Answer_Book/Nature/200p...,1
829282,en.b,Adventist_Youth_Honors_Answer_Book/Nature/300p...,1
829283,en.b,Adventist_Youth_Honors_Answer_Book/Nature/300p...,1
829284,en.b,Adventist_Youth_Honors_Answer_Book/Nature/300p...,1
829285,en.b,Adventist_Youth_Honors_Answer_Book/Nature/300p...,1


# Using Dask

In [3]:
%%time

d1 = dd.read_csv('/data/khodadaa/wiki09-pagecount-logs/wiki09_pagecounts_dumpspagecounts-20090101-110000.csv', 
                 dtype={'lang': str, 'article': str})
d2 = dd.read_csv('/data/khodadaa/wiki09-pagecount-logs/wiki09_pagecounts_dumpspagecounts-20090102-230000.csv',
                dtype={'lang': str, 'article': str})
d3 = dd.read_csv('/data/khodadaa/wiki09-pagecount-logs/wiki09_pagecounts_dumpspagecounts-20090105-230000.csv',
                dtype={'lang': str, 'article': str})

CPU times: user 144 ms, sys: 14 ms, total: 158 ms
Wall time: 164 ms


In [4]:
%%time
j = dd.merge(d1, d2, on=['lang', 'article'], how='outer')

CPU times: user 55.4 ms, sys: 2.55 ms, total: 57.9 ms
Wall time: 56.2 ms


In [9]:
%%time
j.head()

OSError: [Errno 28] No space left on device: '/tmp/tmpp6b_pspj.partd'

# Aggregate counts into one file

In [9]:
import pandas as pd

df = pd.DataFrame(columns=['lang', 'article', 'pagecount-3mon', 'pagecount-norm'])

iter_df = pd.read_csv('/data/khodadaa/wiki09-pagecount-logs/wiki09-images-pagecounts.csv', chunksize=1000)
for chunk in iter_df:    
    chunk['pagecount-3mon'] = chunk.sum(axis=1)
    df = df.append(chunk, ignore_index=True)
    
tot = df['pagecount-3mon'].sum()
df['pagecount-norm'] = df['pagecount-3mon']/tot
df['pagecount-3mon'] = df['pagecount-3mon'].astype(int)
cols_to_keep = ['lang', 'article', 'pagecount-3mon', 'pagecount-norm']
df = df[cols_to_keep]
df[cols_to_keep].to_csv('/data/khodadaa/wiki09-pagecount-logs/wiki09-images-aggregate.csv', index=False)

      lang                                            article  pagecount-3mon  \
0     en.b                File:Tomato_Firmware_-_Overview.PNG            2526   
1       en                          CharlieGlassCover1972.jpg              22   
2       en  File:2003-10-15_1600x900_south_dakota_badlands...            1259   
3       en                           File:240-dogFighting.jpg            9334   
4       en         File:AK-47_type_II_Part_DM-ST-89-01131.jpg           47492   
5       en                                    File:Ah-64d.jpg            6149   
6       en                  File:Alice_In_Chains-Facelift.jpg            3126   
7       en                               File:Ambox_style.png          319898   
8       en                         File:Atmospherelemonsa.jpg            1595   
9       en                         File:Atmospherelemonsb.jpg             278   
10      en                             File:Autoroute_M25.jpg            2615   
11      en                Fi

# Parallel Aggregate

In [1]:
%%time
import multiprocessing as mp
import pandas as pd

# df = pd.DataFrame(columns=['lang', 'article', 'pagecount-3mon', 'pagecount-norm'])

iter_df = pd.read_csv('/data/khodadaa/wiki09-pagecount-logs/wiki09-images-pagecounts.csv', chunksize=1000, nrows=10000)

def process_chunk(chunk):
    chunk['pagecount-3mon'] = chunk.sum(axis=1)
    print('.')
    return chunk[['lang', 'article', 'pagecount-3mon']]
def show_progress(c):
    print('.',end='')
    return c
# df = df.append(chunk, ignore_index=True)
# for chunk in iter_df:
pool = mp.Pool(processes=6)
dfs = pool.map(process_chunk, [c for c in iter_df])
df = pd.concat(dfs)
    
tot = df['pagecount-3mon'].sum()
df['pagecount-norm'] = df['pagecount-3mon']/tot
df['pagecount-3mon'] = df['pagecount-3mon'].astype(int)
cols_to_keep = ['lang', 'article', 'pagecount-3mon', 'pagecount-norm']
df = df[cols_to_keep]
print(df[:5])
# df[cols_to_keep].to_csv('/data/khodadaa/wiki09-pagecount-logs/wiki09-images-aggregate.csv', index=False)

.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
.
   lang                                            article  pagecount-3mon  \
0  en.b                File:Tomato_Firmware_-_Overview.PNG            2526   
1    en                          CharlieGlassCover1972.jpg              22   
2    en  File:2003-10-15_1600x900_south_dakota_badlands...            1259   
3    en                           File:240-dogFighting.jpg            9334   
4    en         File:AK-47_type_II_Part_DM-ST-89-01131.jpg           47492   

   pagecount-norm  
0    7.068041e-05  
1    6.155855e-07  
2    3.522828e-05  
3    2.611762e-04  
4    1.328881e-03  
CPU times: user 7.1 s, sys: 807 ms, total: 7.91 s
Wall time: 10.6 s


In [4]:
%%time
import pandas as pd

# df = pd.DataFrame(columns=['lang', 'article', 'pagecount-3mon'])

iter_df = pd.read_csv('/data/khodadaa/wiki09-pagecount-logs/wiki09-images-pagecounts.csv', chunksize=1000, nrows=10000)
dfs = []
for chunk in iter_df:    
    chunk['pagecount-3mon'] = chunk.sum(axis=1)
    dfs.append(chunk[['lang', 'article', 'pagecount-3mon']])
df = pd.concat(dfs)
    
tot = df['pagecount-3mon'].sum()
df['pagecount-norm'] = df['pagecount-3mon']/tot
df['pagecount-3mon'] = df['pagecount-3mon'].astype(int)
cols_to_keep = ['lang', 'article', 'pagecount-3mon', 'pagecount-norm']
df = df[cols_to_keep]
print(df[:5])
# df[cols_to_keep].to_csv('/data/khodadaa/wiki09-pagecount-logs/wiki09-images-aggregate.csv', index=False)

   lang                                            article  pagecount-3mon  \
0  en.b                File:Tomato_Firmware_-_Overview.PNG            2526   
1    en                          CharlieGlassCover1972.jpg              22   
2    en  File:2003-10-15_1600x900_south_dakota_badlands...            1259   
3    en                           File:240-dogFighting.jpg            9334   
4    en         File:AK-47_type_II_Part_DM-ST-89-01131.jpg           47492   

   pagecount-norm  
0    7.068041e-05  
1    6.155855e-07  
2    3.522828e-05  
3    2.611762e-04  
4    1.328881e-03  
CPU times: user 2min 50s, sys: 5.72 s, total: 2min 56s
Wall time: 26.7 s


# Panda to MySQL

In [75]:
import MySQLdb
import sqlalchemy
import getpass
db_param = {}
db_param['user'] = input('Username: ') or 'khodadaa'
db_param['pass'] = getpass.getpass('Password: ')
db_param['host'] = input('Host: ') or 'mysql.cs.orst.edu'
db_param['dbname'] = input('Database name: ') or 'khodadaa'
db_param['tbl_name'] = input('Table name: ') or 'tbl_temp'
df = pd.read_csv('/data/khodadaa/wiki09-pagecount-logs/wiki09-images-pagecounts.csv')
connect_str = 'mysql+mysqldb://{user}:{pass}@{host}/{dbname}?charset=utf8mb4'.format(**db_param)
df.to_sql(con=connect_str, name=db_param['tbl_name'], if_exists='fail', chunksize=10000)

Username: 
Password: ········
Host: 
Database name: 
Table name: 


# MySQL to Panda

In [3]:
%%time
import MySQLdb
import sqlalchemy
import getpass
db_param = {}
db_param['user'] = input('Username: ') or 'khodadaa'
db_param['pass'] = getpass.getpass('Password: ')
db_param['host'] = input('Host: ') or 'mysql.cs.orst.edu'
db_param['dbname'] = input('Database name: ') or 'khodadaa'
db_param['tbl_name'] = input('Table name: ') or 'tbl_image_09'
connect_str = 'mysql+mysqldb://{user}:{pass}@{host}/{dbname}?charset=utf8mb4'.format(**db_param)
df_sql = pd.read_sql('SELECT * FROM '+ db_param['tbl_name'], con=connect_str)
print(df_sql.shape[0], list(df_sql.columns))

Username: 
Password: ········
Host: 
Database name: 
Table name: 
1183070 ['id', 'src', 'src_tokenized', 'caption', 'popularity']
CPU times: user 8.68 s, sys: 895 ms, total: 9.57 s
Wall time: 13.5 s


# Page count matching

In [4]:
%%time
df_csv = pd.read_csv('/data/khodadaa/wiki09-pagecount-logs/wiki09-images-aggregate.csv')
print(df_csv.shape[0], list(df_csv.columns))

7661938 ['lang', 'article', 'pagecount-3mon', 'pagecount-norm']
CPU times: user 15 s, sys: 919 ms, total: 15.9 s
Wall time: 10.5 s


In [8]:
%%time
import re
regex_pat = re.compile(r'.*:', re.IGNORECASE)
df_csv['src'] = df_csv['article'].str.replace(regex_pat, '')

CPU times: user 2min, sys: 1.09 s, total: 2min 2s
Wall time: 1min 56s


In [9]:
df_csv[:10]

,lang,article,pagecount-3mon,pagecount-norm,src
0,en.b,File:Tomato_Firmware_-_Overview.PNG,2526,2.954064e-06,Tomato_Firmware_-_Overview.PNG
1,en,CharlieGlassCover1972.jpg,22,2.572819e-08,CharlieGlassCover1972.jpg
2,en,File:2003-10-15_1600x900_south_dakota_badlands...,1259,1.472354e-06,2003-10-15_1600x900_south_dakota_badlands.jpg
3,en,File:240-dogFighting.jpg,9334,1.091577e-05,240-dogFighting.jpg
4,en,File:AK-47_type_II_Part_DM-ST-89-01131.jpg,47492,5.554014e-05,AK-47_type_II_Part_DM-ST-89-01131.jpg
5,en,File:Ah-64d.jpg,6149,7.191028e-06,Ah-64d.jpg
6,en,File:Alice_In_Chains-Facelift.jpg,3126,3.655741e-06,Alice_In_Chains-Facelift.jpg
7,en,File:Ambox_style.png,319898,3.741089e-04,Ambox_style.png
8,en,File:Atmospherelemonsa.jpg,1595,1.865293e-06,Atmospherelemonsa.jpg
9,en,File:Atmospherelemonsb.jpg,278,3.251107e-07,Atmospherelemonsb.jpg


In [11]:
%%time
df_csv_file = df_csv[df_csv['lang'] == 'en']
df_csv_file = df_csv_file[df_csv_file['article'].str.lower().str.startswith('file:')]
print(df_csv_file.shape[0])

2481417
CPU times: user 17.6 s, sys: 1.3 s, total: 18.9 s
Wall time: 8.61 s


In [12]:
%%time
df_mg = pd.merge(df_sql, df_csv_file, on='src', how='inner')
print(df_sql.shape[0],df_mg.shape[0])

1183070 1095199
CPU times: user 9.25 s, sys: 625 ms, total: 9.88 s
Wall time: 3.78 s


In [15]:
df_mg = df_mg[['src', 'src_tokenized', 'caption', 'popularity', 'pagecount-3mon', 'pagecount-norm']]
df_mg[:10]

,src,src_tokenized,caption,popularity,pagecount-3mon,pagecount-norm
0,David_Suchet_-_Poirot.png,David Suchet Poirot,,89770,1508,1.763550e-06
1,The_Labours_of_Hercules.jpg,The Labours of Hercules,,89770,253,2.958741e-07
2,Ustinov_is_Poirot.jpg,Ustinov is Poirot,,89770,260,3.040604e-07
3,Poirot_Season_11_DVD.jpg,Poirot Season 11 DVD,,95144,799,9.344009e-07
4,Disambig_gray.svg,Disambig gray,,329049600,47062,5.503727e-05
5,Regione-Lombardia-Stemma.svg,Regione Lombardia Stemma,,829953,463,5.414614e-07
6,Star_full.svg,Star full,,176909636,55002,6.432280e-05
7,Star_empty.svg,Star empty,,146047141,25404,2.970904e-05
8,Star_half.svg,Star half,,91318633,7019,8.208461e-06
9,Mokopa_(Impi).jpg,Mokopa Impi,,2534,121,1.415050e-07


In [16]:
%%time
connect_str = 'mysql+mysqldb://{user}:{pass}@{host}/{dbname}?charset=utf8mb4'.format(**db_param)
df_mg.to_sql(con=connect_str, name='tbl_image_09_pagecount', if_exists='fail', index=False)

CPU times: user 52.2 s, sys: 2.66 s, total: 54.8 s
Wall time: 40.3 s
